In [1]:
import numpy as np

from torch.utils.data import DataLoader
from torchvision import datasets

from tqdm.notebook import tqdm

import MST

In [2]:
BATCH_SIZE = 32
NUM_WORKERS = 0

EPOCHS = 10

In [3]:
def to_np_arr(a):
    return np.array(a) / 255

In [4]:
transform = to_np_arr

train_dataset = datasets.MNIST(
    root='datasets',
    train=True,
    transform=transform,
    download=True
)

test_dataset = datasets.MNIST(
    root='datasets',
    train=False,
    transform=transform,
    download=True
)

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
)

In [5]:
class MyNet(MST.BasicModule):
    def __init__(self):
        super().__init__()
# caiming ke 
# resnet 
        self.net_blocks = MST.Sequential(

            MST.FullyConnectedLayer(28*28, 100),
            MST.ReLu(),

            MST.FullyConnectedLayer(100, 50),
            MST.ReLu(),

            MST.FullyConnectedLayer(50, 10),
            MST.ReLu()
        )

    def forward(self, x):
        x = x.reshape(-1, 28*28)
        x = self.net_blocks(x)
        return x

    def backward(self, dOut):
        dN = self.net_blocks.backward(dOut)
        return dN

CELoss = MST.CrossEntropyLoss()
net = MyNet()

optimizer = MST.SGD(net, lr=0.01, momentum=0.9)
print(net)

MyNet:
└── net_blocks: 
	 └── FullyConnectedLayer: Trainable(True)
	 └── ReLu: Trainable(False)
	 └── FullyConnectedLayer: Trainable(True)
	 └── ReLu: Trainable(False)
	 └── FullyConnectedLayer: Trainable(True)
	 └── ReLu: Trainable(False)


In [6]:
def train(net : MST.BasicModule, optimizer : MST.SGD, criterion : MST.BasicModule):
    running_loss = 0
    for ind, (images, labels) in enumerate(train_dataloader):
        images = np.array(images)
        labels = np.array(labels)
        
        output = net(images)
        loss = criterion(output, labels)
        net.backward(criterion.backward())
        optimizer.step()

        running_loss += loss
    train_loss = running_loss / len(train_dataloader)
    return train_loss


def valid(net : MST.BasicModule, criterion : MST.BasicModule):
    running_loss = 0
    correct_total = 0
    for images, labels in test_dataloader:
        labels = np.array(labels)

        output = net(images)

        loss = criterion(output, labels)
        running_loss += loss

        pred = np.argmax(output, axis=1, keepdims=True)
        correct_total += np.sum(pred[pred==labels.reshape(-1, 1)].astype(bool))
        
    rec = correct_total / len(test_dataloader.dataset)
    valid_loss = running_loss / len(test_dataloader)
    return valid_loss, rec

In [7]:
for epoch in (pbar := tqdm(range(EPOCHS))):
    train_loss = train(net, optimizer, CELoss)
    valid_loss, rec = valid(net, CELoss)

    print(f"[{epoch}] train/valid loss: {train_loss:.4f}/{valid_loss:.4f} rec: {rec:.4f}")
    pbar.set_description(f"train/valid loss: {train_loss:.4f}/{valid_loss:.4f} rec: {rec:.4f}")

  0%|          | 0/10 [00:00<?, ?it/s]

[0] train/valid loss: 0.5215/0.1872 rec: 0.8511
[1] train/valid loss: 0.1469/0.1279 rec: 0.8663
[2] train/valid loss: 0.1014/0.1218 rec: 0.8685
[3] train/valid loss: 0.0790/0.1164 rec: 0.8694
[4] train/valid loss: 0.0653/0.1035 rec: 0.8729
[5] train/valid loss: 0.0521/0.1109 rec: 0.8728
[6] train/valid loss: 0.0433/0.1028 rec: 0.8737
[7] train/valid loss: 0.0351/0.1003 rec: 0.8758
[8] train/valid loss: 0.0290/0.1082 rec: 0.8740
[9] train/valid loss: 0.0238/0.0993 rec: 0.8775
